In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

from sklearn.svm import SVC ## 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/ManonYa09/Statistics_with_Python_G7/main/Part%204%20Regression%20Analysis/Logistics%20Regression/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [7]:
columns_to_drop = ['customerID', 'gender', 'TotalCharges']
yes_no_columns = []
categorical_columns= []
numerical_columns = []
target = 'Churn'
for column in df.columns:
    if  column == target:
        continue
    elif column in columns_to_drop:
        continue
    elif df[column].nunique() == 2:
        yes_no_columns.append(column)
    elif df[column].nunique() <=4:
        categorical_columns.append(column)
    else:
        numerical_columns.append(column)

In [50]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, FunctionTransformer
from sklearn.linear_model import LogisticRegression

In [52]:
def drop_columns(X):
    return X.drop(columns=columns_to_drop)

In [54]:
preprocessing = ColumnTransformer([
    ('One_hot', OneHotEncoder(), categorical_columns),
    ('yes_no', OrdinalEncoder(), yes_no_columns),
    ('scaling', StandardScaler(), numerical_columns)])

In [56]:
preprocessing = ColumnTransformer([
    ('One_hot', OneHotEncoder(), categorical_columns),
    ('yes_no', OrdinalEncoder(), yes_no_columns),
    ('scaling', StandardScaler(), numerical_columns)])

In [58]:
traning_pipeline_logistic = Pipeline([
    ('drop', FunctionTransformer(drop_columns)),
    ('preproccessing', preprocessing),
    ('model1', LogisticRegression())
    ])

In [64]:
x = df.drop(columns=target)
y = df[target]
x_train, x_test, y_train , y_test = train_test_split(x, y, test_size=.1, random_state=12) ## cross validation

In [70]:
traning_pipeline_logistic.fit(x_train, y_train)

Pipeline(steps=[('drop',
                 FunctionTransformer(func=<function drop_columns at 0x1772374c0>)),
                ('preproccessing',
                 ColumnTransformer(transformers=[('One_hot', OneHotEncoder(),
                                                  ['MultipleLines',
                                                   'InternetService',
                                                   'OnlineSecurity',
                                                   'OnlineBackup',
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies',
                                                   'Contract',
                                                   'PaymentMethod']),
                                                 ('yes_no', OrdinalEncoder(),
                                                  ['SeniorCitizen', 'Partner',
                                                   'Dependents', 'PhoneService',
                                                   'PaperlessBilling']),
                                                 ('scaling', StandardScaler(),
                                                  ['tenure',
                                                   'MonthlyCharges'])])),
                ('model1', LogisticRegression())])

In [72]:
y_prediction = traning_pipeline_logistic.predict(x_test)

In [76]:
from sklearn.metrics import confusion_matrix

In [78]:
confusion_matrix(y_test, y_prediction)

array([[466,  64],
       [ 80,  95]])

In [103]:
traning_pipeline_SVM = Pipeline([
    ('drop', FunctionTransformer(drop_columns)),
    ('preproccessing', preprocessing),
    ('model1', SVC(kernel='poly'))
    ])

In [122]:
param_grid = {
    'model1__kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
      # Including multiple kerne

In [124]:
from sklearn.model_selection import GridSearchCV 

In [126]:
# Perform GridSearchCV
# model_svc = SVC()
grid = GridSearchCV(estimator=traning_pipeline_SVM, param_grid=param_grid, scoring='accuracy')


In [128]:
grid.fit(x_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('drop',
                                        FunctionTransformer(func=<function drop_columns at 0x1772374c0>)),
                                       ('preproccessing',
                                        ColumnTransformer(transformers=[('One_hot',
                                                                         OneHotEncoder(),
                                                                         ['MultipleLines',
                                                                          'InternetService',
                                                                          'OnlineSecurity',
                                                                          'OnlineBackup',
                                                                          'DeviceProtection',
                                                                          'TechSupport',
                                                                          'StreamingTV',
                                                                          'StreamingMovies',
                                                                          'Contract',
                                                                          'PaymentMethod']),
                                                                        ('yes_no',
                                                                         OrdinalEncoder(),
                                                                         ['SeniorCitizen',
                                                                          'Partner',
                                                                          'Dependents',
                                                                          'PhoneService',
                                                                          'PaperlessBilling']),
                                                                        ('scaling',
                                                                         StandardScaler(),
                                                                         ['tenure',
                                                                          'MonthlyCharges'])])),
                                       ('model1', SVC(kernel='poly'))]),
             param_grid={'model1__kernel': ['linear', 'poly', 'rbf',
                                            'sigmoid']},
             scoring='accuracy')

In [134]:
y_prediction_svm = grid.predict(x_test)

In [136]:
confusion_matrix(y_test, y_prediction_svm)

array([[463,  67],
       [ 79,  96]])

In [138]:
grid.best_params_

{'model1__kernel': 'linear'}